In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/Final Project/balanced_data.zip", 'r')

zip_ref.extractall("/tmp")
zip_ref.close()

In [3]:
import os
import keras

Using TensorFlow backend.


In [4]:
!ls "/tmp/balanced_data/"

test  train  val


In [5]:
base_dir = '/tmp/balanced_data/'

#the original dataset interchanged 'test' with 'validation,' with the latter being the 'holdout' dataset
# train_dir = os.path.join(base_dir, 'train')
# test_dir = os.path.join(base_dir, 'test')
# validation_dir = os.path.join(base_dir, 'val')

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [7]:
# dimensions of our images.
img_width, img_height = 150, 150

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 1)

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [9]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical')

Found 2682 images belonging to 2 classes.
Found 468 images belonging to 2 classes.


In [10]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=nb_train_samples // batch_size,
                              epochs=epochs,
                              validation_data=test_generator,
                              validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Epoch 1/50
125/125 [==============================] - 37s 299ms/step - loss: 0.5710 - accuracy: 0.6930 - val_loss: 0.4254 - val_accuracy: 0.8223
Epoch 2/50
125/125 [==============================] - 30s 240ms/step - loss: 0.3751 - accuracy: 0.8531 - val_loss: 0.4166 - val_accuracy: 0.8711
Epoch 3/50
125/125 [==============================] - 31s 247ms/step - loss: 0.3461 - accuracy: 0.8610 - val_loss: 0.5795 - val_accuracy: 0.8106
Epoch 4/50
125/125 [==============================] - 29s 235ms/step - loss: 0.2975 - accuracy: 0.8862 - val_loss: 0.4804 - val_accuracy: 0.8579
Epoch 5/50
125/125 [==============================] - 32s 253ms/step - loss: 0.2755 - accuracy: 0.9017 - val_loss: 0.5112 - val_accuracy: 0.8840
Epoch 6/50
125/125 [==============================] - 30s 243ms/step - loss: 0.2465 - accuracy: 0.9047 - val_loss: 1.4667 - val_accuracy: 0.7951
Epoch 7/50
125/125 [==============================] - 31s 246ms/step - loss: 0.2562 - accuracy: 0.9067 - val_loss: 0.1206 - val_ac

In [11]:
import pandas as pd
import numpy as np

In [ ]:
history_df = pd.DataFrame(history.history)

history_df

In [14]:
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical')


test_generator.reset()

pred = model.predict_generator(val_generator,verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (val_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=val_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

print(results.to_string())

Found 16 images belonging to 2 classes.
1/1 [==============================] - 0s 123ms/step
                                   Filename Predictions
0          NORMAL/NORMAL2-IM-1427-0001.jpeg   PNEUMONIA
1          NORMAL/NORMAL2-IM-1430-0001.jpeg   PNEUMONIA
2          NORMAL/NORMAL2-IM-1431-0001.jpeg   PNEUMONIA
3          NORMAL/NORMAL2-IM-1436-0001.jpeg   PNEUMONIA
4          NORMAL/NORMAL2-IM-1437-0001.jpeg   PNEUMONIA
5          NORMAL/NORMAL2-IM-1438-0001.jpeg   PNEUMONIA
6          NORMAL/NORMAL2-IM-1440-0001.jpeg   PNEUMONIA
7          NORMAL/NORMAL2-IM-1442-0001.jpeg   PNEUMONIA
8   PNEUMONIA/person1946_bacteria_4874.jpeg      NORMAL
9   PNEUMONIA/person1946_bacteria_4875.jpeg   PNEUMONIA
10  PNEUMONIA/person1947_bacteria_4876.jpeg   PNEUMONIA
11  PNEUMONIA/person1949_bacteria_4880.jpeg   PNEUMONIA
12  PNEUMONIA/person1950_bacteria_4881.jpeg   PNEUMONIA
13  PNEUMONIA/person1951_bacteria_4882.jpeg   PNEUMONIA
14  PNEUMONIA/person1952_bacteria_4883.jpeg   PNEUMONIA
15  PNEUMON

In [20]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import model_from_json

# save_path = '/content/drive/My Drive/Colab Notebooks/Final Project/models/'

dot_img_file = '/content/drive/My Drive/Colab Notebooks/Final Project/images/Chollet_balanced.png'
plot_model(model, to_file=dot_img_file, show_shapes=True)

results.to_csv('/content/drive/My Drive/Colab Notebooks/Final Project/models/chollet_balanced/chollet_bal_eval.csv', index=False)
history_df.to_csv('/content/drive/My Drive/Colab Notebooks/Final Project/models/chollet_balanced/chollet_bal_acc.csv', index=False)

model_json = model.to_json()
with open('/content/drive/My Drive/Colab Notebooks/Final Project/models/chollet_balanced/chollet_bal_model.json', 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('/content/drive/My Drive/Colab Notebooks/Final Project/models/chollet_balanced/chollet_bal_model.h5')